# 21_ 카카오 OpenAPI를 연동한 이미지 수집(2)

전체 처리 루틴을 반복문으로 감싸면서 페이지 번호에 해당하는 변수만 변경하면 1페이지부터 50페이지까지 일괄 수집이 가능하다.

## #01. 기본 준비 단계

### 1) 필요한 패키지 가져오기 

In [21]:
import requests
import json
import urllib
import os
import datetime as dt
import pandas as pd
from pandas import DataFrame

### 2) Open API 연동키 정의 

In [22]:
api_key = "681e20ed293a6f6822156c7de5162153"

### 3) 검색어 및 페이지 구현에 필요한 변수 준비 

In [23]:
q = "박보영"
page = 1 
size = 80

### 4) 이미지가 저장될 폴더의 이름 만들기 

#### 검색어 + 연재 날짜 형식의 폴더 이름 준비

In [24]:
datetime = dt.datetime.now().strftime("%y%m%d_%H%M%S")
dirname = "%s_%s"  %(q, datetime)
dirname

'박보영_200328_115515'

#### 폴더 생성

In [25]:
if not os.path.exists(dirname):
    os.mkdir(dirname)

## #02.Open API 연동

### 1) 접속 세션 만들기 

In [26]:
# 접속 세션 만들기 
user_agent = "Mozilla/5.0 (Linux; Android 5.0; SM-G900P Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Mobile Safari/537.36"

session = requests.Session()

### 2) 세션의 HTTP 헤더 설정
인증키를 header에 포함 시켜야 함  "KaKaoAK" 뒤에 공백 추가 주의 

In [27]:
session.headers.update({'User-agent':user_agent, 'referer':None, 'Authorization':'KakaoAK '+api_key})

## #03. 반복문을 수집하면서 이미지 수집 

페이지 번호 변수 (`page`)를 반복문에 의해 변경하면서 계속햬서 Open API 연동을 시도한다.

In [37]:
count = 0 

for i in range(0,10) :   # 0~9까지 --> 반복 범위를 조절하면 최대 다운받을 이미지 수가 제어된다.
    page = i + 1         #  1~10까지가 된다.
    
    # API에 전달할 파라미터 인코딩
    params = {"page":page, "size":size,"query":q}
    query = urllib.parse.urlencode(params)
    # print(query)
    
    # 최종 접속 주소 구성
    url_tpl = "https://dapi.kakao.com/v2/search/image"
    api_url = url_tpl + "?" + query
    # print(api_url)
    
    # API에 접근하여 데이터 가져오기
    r = session.get(api_url)

    if r.status_code != 200 :
        print("[%d Error] %s" % (r.status_code, r.reason))
        continue
    
    # 가져온 결과를 딕셔너리로 변환
    r.encoding = "utf-8"
    image_dict = json.loads(r.text)
    # print(image_dict)
    
    # DataFrame 생성
    image_df = DataFrame(image_dict['documents'])
    # print(image_df)
    
    # 이미지 주소에 대해서만 반복
    for image_url in image_df['image_url']:
        # 카운터 증가 
        count += 1
    
        # 파일이 저장될 경로 생성
        path = "%s/%04d.jpg" %(dirname, count)
        print("[%s] >> %s" % (path, image_url))
    
        try:
            # 이미지 주소를 다운로드를 위해  stream 모드로 가져온다.
            r = session.get(image_url, stream=True)
        
            # 에러 발생시 저장이 불가능하므로 건너뛰고 반복의 조건식으로 이동
            if r.status_code != 200 :
                print("######> 저장실패1")
                continue
            
            ## 추출한 데이터를 저장
            # -> 'w' : 텍스트 쓰기 모드, 'wb':바이너리(이진값) 쓰기 모드
            with open(path, 'wb') as f:
                f.write(r.raw.read())
                print("------> 저장 성공")
        except Exception as ex:
            print("######> 저장실패2")
            print(ex)  # 에러 메세지를 강제 출력 --> 에러 원인을 확인 하기 위함.
            continue

[박보영_200328_115515/0001.jpg] >> http://cfile117.uf.daum.net/image/23500A4950DAF50A154621
------> 저장 성공
[박보영_200328_115515/0002.jpg] >> http://t1.daumcdn.net/news/201506/29/ned/20150629145904910pxuu.php
------> 저장 성공
[박보영_200328_115515/0003.jpg] >> http://cfile76.uf.daum.net/image/2236014F524AD46232F77B
------> 저장 성공
[박보영_200328_115515/0004.jpg] >> http://t1.daumcdn.net/news/201508/23/ned/20150823183225453txjv.jpg
------> 저장 성공
[박보영_200328_115515/0005.jpg] >> http://t1.daumcdn.net/news/201508/23/ned/20150823185242035faex.jpg
------> 저장 성공
[박보영_200328_115515/0006.jpg] >> http://t1.daumcdn.net/news/201510/14/ned/20151014160318622ectb.jpg
------> 저장 성공
[박보영_200328_115515/0007.jpg] >> http://t1.daumcdn.net/news/201510/03/ned/20151003130602601rsch.jpg
------> 저장 성공
[박보영_200328_115515/0008.jpg] >> http://www.viva100.com/mnt/images/file/2019y/08m/27d/20190827002305505_1.jpg
------> 저장 성공
[박보영_200328_115515/0009.jpg] >> http://t1.daumcdn.net/news/201505/21/ned/20150521120102503zpvv.php
------> 